<a href="https://colab.research.google.com/github/sssanghn/AI-Emotion-Diary/blob/main/KoBERT_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT 감정 다중분류모델

> KoBERT를 이용하여 한국어 문장을 여러 클래스로 분류하는 모델을 만들어 보려고 한다.
</br>
공포, 놀람, 분노, 슬픔, 중립, 기쁨, 혐오와 같은 감정이 느껴지는 짧은 대화 텍스트를 각각 어떠한 감정의 텍스트인지 분류하는 모델을 만드는 것이다. </br>
AIHUB에서 가져온 데이터셋으로 KoBERT 모델을 FineTuning 하여 </br>
최종적으로 AI 감정일기 어플리케이션에서 작성한 일기 텍스트에서 감정을 추출하기 위해 프로젝트를 진행하였다.

**KoBERT란?**

> BERT는 약 33억 개의 단어로 pretrain 되어 있는 기계번역 모델이다. </br>
하지만 외국에서 만든 것이다 보니 영어에 대해서는 정확도는 높고, 한국어에 대해서는 정확도가 떨어진다.</br>
따라서 BERT 모델을 한국어에도 잘 활용할 수 있도록 만들어진 것이 KoBERT이다. </br>
KoBERT는 BERT 모델에서 한국어 데이터를 추가로 학습시킨 모델로, 한국어 위키에서 5백만개의 문장과 5천4백만개의 단어를 학습시킨 모델이다. </br>
따라서 한국어 데이터에 대해서도 높은 정확도를 낼 수 있다고 한다. </br>
한국어의 불규칙한 언어 변화의 특성을 반영하기 위해 데이터 기반 토큰화 (Tokenization) 기법을 적용하여 기존 대비 27%의 토큰만으로 2.6% 이상의 성능 향상을 이끌어 낸 모델이다. </br>

# 1. 프로젝트 설명

**KoBERT 파인튜닝**
> BERT 모델은 자신의 사용 목적에 따라 파인튜닝(FineTuning)이 가능하다. </br>
따라서 Output layer만 추가로 달아주면 원하는 결과를 출력해내는 기계번역 모델을 만들어 낼 수 있다. </br>
이 점을 활용하여 KoBERT 모델을 FineTuning하여 최종적으로 일기 텍스트에서 감정을 추출해내는 모델을 새로 만들어 보려고 한다.

# 2. 데이터 수집

**데이터 수집**
> 이번 프로젝트를 위해 사용한 한국어 대화 문장 데이터셋은 총 4개이다. </br>
</br>
1. 한국어 감정 정보가 포함된 단발성 대화 데이터셋
* SNS 글 및 온라인 댓글에 대한 웹 크롤링을 실시하여 선정된 AIHUB의 공공 데이터셋
* 약 3만 9천건의 데이터셋(기쁨, 슬픔, 놀람, 분노, 공포, 혐오, 중립의 7개 감정)
</br>
</br>
2. 한국어 감정 정보가 포함된 연속적 대화 데이터셋
* SNS 글 및 온라인 댓글에 대한 웹 크롤링을 실시하여 선정된 AIHUB의 공공 데이터셋
* 연속적 10,000개 대화세트, 단발성으로는 55,627 문장 데이터셋(기쁨, 슬픔, 놀람, 분노, 공포, 혐오, 중립의 7개 감정)
</br>
</br>
3. 감정 분류를 위한 대화 음성 데이터셋
* AI HUB내 KAIST 인공지능연구소에서 구축한 감정 분류를 위한 대화 음성 데이터셋에서 '5차년도 2차 CSV'를 활용
* 감성대화 어플리케이션을 이용하여 수집한 AIHUB의 공공 데이터셋
* 일정 기간동안 사용자들이 어플리케이션과 자연스럽게 대화하고, 수집된 데이터를 정제 작업을 거쳐 선별
* 7가지 감정(기쁨, 슬픔, 놀람, 분노, 공포, 혐오, 중립의 7개 감정)
</br>
</br>


# Colab 환경 설정

> 모델을 구현하기에 앞서 코랩 환경설정을 진행하려고 한다. </br>
해당 사양은 KoBERT 오픈소스 내 requirements.txt를 참고하였다.

In [6]:
# Colab 환경 설정
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

  Using cached sentencepiece-0.1.91.tar.gz (500 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached transformers-4.8.2-py3-none-any.whl (2.5 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did no

> 다음으로 깃허브 내 파일을 Colab으로 가져온다.

In [ ]:
# https://github.com/SKTBrain/KoBERT의 파일들을 Colab으로 다운로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

> 그 다음으로는 해당 라이브러리들을 import 해준다.

In [ ]:
# KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd